In [1]:
#| default_exp encoder

In [2]:
#| hide
#%load_ext autoreload --> Not working TODO:REVISAR
# %autoreload 2 

In [3]:
#| export
from dvats.memory import *

In [1]:
## -- Classes & types
from dataclasses import dataclass, field
from typing import List, Optional, Tuple, Callable

# Encoder

> Architectures and functions for creating encoders that create the embeddings

In [4]:
#| export
import pandas as pd
import numpy as np
from fastcore.all import *
from tsai.callback.MVP import *
from tsai.imports import *
from tsai.models.InceptionTimePlus import InceptionTimePlus
from tsai.models.explainability import get_acts_and_grads
from tsai.models.layers import *
from tsai.data.validation import combine_split_data
import time

In [5]:
#| hide
from tsai.all import *

### Architectures

In [6]:
#|export 
class DCAE_torch(Module):
    def __init__(self, c_in, seq_len, delta, nfs=[64, 32, 12], kss=[10, 5, 5],
                 pool_szs=[2,2,3], output_fsz=10):
        """
        Create a Deep Convolutional Autoencoder for multivariate time series of `d` dimensions,
        sliced with a window size of `w`. The parameter `delta` sets the number of latent features that will be
        contained in the Dense layer of the network. The the number of features
        maps (filters), the filter size and the pool size can also be adjusted."
        """
        assert all_equal([len(x) for x in [nfs, kss, pool_szs]], np.repeat(len(nfs), 3)), \
            'nfs, kss, and pool_szs must have the same length'
        assert np.prod(pool_szs) == nfs[-1], \
            'The number of filters in the last conv layer must be equal to the product of pool sizes'
        assert seq_len % np.prod(pool_szs) == 0, \
            'The product of pool sizes must be a divisor of the window size'
        layers = []
        for i in range_of(kss):
            layers += [Conv1d(ni=nfs[i-1] if i>0 else c_in, nf=nfs[i], ks=kss[i]),
                       nn.MaxPool1d(kernel_size=pool_szs[i])]
        self.downsample = nn.Sequential(*layers)
        self.bottleneck = nn.Sequential(OrderedDict([
            ('flatten', nn.Flatten()),
            ('latent_in', nn.Linear(seq_len, delta)),
            ('latent_out', nn.Linear(delta, seq_len)),
            ('reshape', Reshape(nfs[-1], seq_len // np.prod(pool_szs)))
        ]))
        layers = []
        for i in reversed(range_of(kss)):
            layers += [Conv1d(ni=nfs[i+1] if i != (len(nfs)-1) else nfs[-1],
                              nf=nfs[i], ks=kss[i]),
                       nn.Upsample(scale_factor=pool_szs[i])]
        layers += [Conv1d(ni=nfs[0], nf=c_in, kernel_size=output_fsz)]
        self.upsample = nn.Sequential(*layers)


    def forward(self, x):
        x = self.downsample(x)
        x = self.bottleneck(x)
        x = self.upsample(x)
        return x

In [7]:
#| hide
foo = torch.rand(3, 1, 48)
m = DCAE_torch(c_in=foo.shape[1], seq_len=foo.shape[2], delta=12)
m(foo).shape

torch.Size([3, 1, 48])

### Dictionary to get the default backbone modules to get the embeddings from

In [8]:
#| export
ENCODER_EMBS_MODULE_NAME = {
    InceptionTimePlus: 'backbone', # for mvp based models
    DCAE_torch: 'bottleneck.latent_in'
}

### Getting the embeddings (activations) from the encoder

In [ ]:
#| export
from fastai.learner import Learner
from tsai.data.core import TSDataLoaders

In [ ]:
#| export
def get_enc_embs_ensure_batch_size_(
    dls        : TSDataLoaders,
    batch_size : int = None,
    verbose    : int = 0
) -> None:
    if batch_size is None:
        if verbose > 1: 
            print("[ Get Encoder Embeddings Ensure Batch Size ] No batch size proposed")
        if dls.bs == 0: 
            if verbose > 1: 
                print("[ Get Encoder Embeddings Ensure Batch Size ] Using value 64 as 0 is not a valid value.")
            enc_learn.dls.bs = 64
        elif verbose > 1: 
            print(f"[ Get Encoder Embeddings Ensure Batch Size ] Using the original value: {dls.bs}")
    else:
        dls.bs = batch_size
        if verbose > 1: 
            print(f"[ Get Encoder Embeddings Ensure Batch Size ] Batch size proposed. Using {dls.bs}")

In [26]:
#| export
from fastai.learner import Learner
from tsai.data.core import TSDataLoaders

In [32]:
#| export
def get_enc_embs_ensure_batch_size_(
    dls        : TSDataLoaders,
    batch_size : int = None,
    verbose    : int = 0
) -> None:
    if batch_size is None:
        if verbose > 1: 
            print("[ Get Encoder Embeddings Ensure Batch Size ] No batch size proposed")
        if dls.bs == 0: 
            if verbose > 1: 
                print("[ Get Encoder Embeddings Ensure Batch Size ] Using value 64 as 0 is not a valid value.")
            enc_learn.dls.bs = 64
        elif verbose > 1: 
            print(f"[ Get Encoder Embeddings Ensure Batch Size ] Using the original value: {dls.bs}")
    else:
        dls.bs = batch_size
        if verbose > 1: 
            print(f"[ Get Encoder Embeddings Ensure Batch Size ] Batch size proposed. Using {dls.bs}")

In [28]:
#| export
def get_enc_embs(
    X               : List [ List [ List [ float ] ] ], 
    enc_learn       : Learner, 
    module          : str  = None, 
    cpu             : bool = False, 
    average_seq_dim : bool = True, 
    to_numpy        : bool = True,
    batch_size      : int  = None,
    verbose         : int  = 0
):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
        - `batch_size`: force data loader to use the input batch size
        - `verbose`: print flag. More big, more information.
    """
    
    if cpu:
        if verbose > 0: print("[ Get Encoder Embeddings ] CPU")
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        if verbose > 0: print("[ Get Encoder Embeddings ] --> GPU")
        if verbose > 1: print("[ Get Encoder Embeddings ] GPU | Ensure empty cache")
        torch.cuda.empty_cache()
        if verbose > 1: print("[ Get Encoder Embeddings ] GPU | Move & exec into CUDA")
        enc_learn.dls.cuda()
        enc_learn.cuda()
        if torch.cuda.is_available():
            if verbose > 1: 
                print("[ Get Encoder Embeddings ] GPU | CUDA is available")
                print(f"[ Get Encoder Embeddings ] GPU | CUDA is available | current device id {torch.cuda.current_device()}")
                print(f"[ Get Encoder Embeddings ] GPU | CUDA is available | current device name {torch.cuda.get_device_name(torch.cuda.current_device())}")            
        else:
            if verbose > 1: print("[ Get Encoder Embeddings ] GPU | CUDA is not available")
        if verbose > 0: print("[ Get Encoder Embeddings ] GPU -->")

    #if verbose > 0: print("[ Get Encoder Embeddings ] Ensure the correct batch size")
    #get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)
    
    if verbose > 0: print("[ Get Encoder Embeddings ] Set dataloader from X (enc_learn does not contain dls)")
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    get_enc_embs_ensure_batch_size_(aux_dl, batch_size, verbose)
    if verbose > 0: print("[ Get Encoder Embeddings ] Get module")
    module = nested_attr(enc_learn.model,ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) if module is None else module
    
    if verbose > 0: print("[ Get Encoder Embeddings ] get_acts_and_grads ")
    if verbose > 1: print(f"[ Get Encoder Embeddings ] get_acts_and_grads bs = {aux_dl.bs}")
    
    embs = [
        get_acts_and_grads(
            model   = enc_learn.model,
            modules = module,
            x       = xb[0], 
            cpu     = cpu
        )[0] 
        for xb in aux_dl
    ]
    if verbose > 0: print("[ Get Encoder Embeddings ] get_acts_and_grads | --> Concat")
    if not cpu:
        if verbose > 1: print("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory")
        total_emb_size = sum([emb.element_size() * emb.nelement() for emb in embs])
        free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()
        if (total_emb_size < free_memory):
            if verbose > 1: print("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory | Fits in GPU -> Computing in GPU")
            embs=[emb.cuda() for emb in embs]
        else:
            if verbose > 1: print("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory | Does not fit in GPU -> Computing in CPU")
            embs=[emb.cpu() for emb in embs]
    if verbose > 1: print("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | to_concat")
    embs = to_concat(embs)
    if verbose > 0: print("[ Get Encoder Embeddings ] get_acts_and_grads | Concat -->")
    
    if verbose > 0: print("[ Get Encoder Embeddings ] Reduce to 2 dimensions.")
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    if verbose > 0: print("[ Get Encoder Embeddings ] Ensure CPU saving & numpy format")
    if to_numpy: embs = embs.numpy() if cpu else embs.cpu().numpy()
    return embs

In [10]:
#| export

def get_enc_embs_set_stride_set_batch_size(
    X                  : List [ List [ List [ float ] ] ], 
    enc_learn          : Learner, 
    stride             : int, 
    batch_size         : int, 
    module             : str  = None, 
    cpu                : bool = False, 
    average_seq_dim    : bool = True, 
    to_numpy           : bool = True, 
    verbose            : int  = 0, 
    time_flag          : bool = False, 
    chunk_size         : int  = 0, 
    check_memory_usage : bool = False
):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `X`: encoder input
        - `enc_learn`: trained encoder
        - `stride`: stride used for the training. Neccesary for adjusting the encoder input
        - `batch_size`: value to force the dataloader to use.
        - `module`: for geting the embeddings of an specific layer.
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
        - `verbose`: For printing messages. More big, more messages.
        - `time_flag`: To take note of the execution time required by this function
        - `chunk_size`: For spliting the embedings reading in batches of `chunk_size` size.
        - `check_memory_usage`: For showing messages of the current state of the memory.
    """
    if time_flag:
        t_start = time.time()
    if verbose > 0:
        print("--> get_enc_embs_set_stride_set_batch_size")
    if check_memory_usage: gpu_memory_status()
    X = X[::stride]
    enc_learn.dls.bs = batch_size 

    get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)
    
    if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | Check CUDA | X ~ ", X.shape[0])
    if cpu:
        if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | Get enc embs CPU")
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        if torch.cuda.is_available():
            if verbose > 0: 
                print("get_enc_embs_set_stride_set_batch_size | CUDA device id:", torch.cuda.current_device())
                print("get_enc_embs_set_stride_set_batch_size | CUDA device name: ", torch.cuda.get_device_name(torch.cuda.current_device()))
                print("get_enc_embs_set_stride_set_batch_size | Ensure empty cache & move 2 GPU")
            torch.cuda.empty_cache()
            enc_learn.dls.cuda()
            enc_learn.cuda()
        else:
            if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | No cuda available. Set CPU = true")
            cpu = True
            
    get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)

    if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | Set dataset from X (enc_learn does not contain dls)")
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    aux_dl.bs = enc_learn.dls.bs if enc_learn.dls.bs>0 else 64
    if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | Get module")
    module = nested_attr(enc_learn.model,ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) if module is None else module
    
    if verbose > 0: 
        #print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | module ", module)
        print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl len", len(aux_dl))
        print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl.batch_len ", len(next(iter(aux_dl))))
        print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl.bs ", aux_dl.bs)
        if (not cpu):
            total = torch.cuda.get_device_properties(device).total_memory
            used = torch.cuda.memory_allocated(torch.cuda.current_device())
            reserved = torch.cuda.memory_reserved(torch.cuda.current_device())
            print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | total_mem ", total)
            print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | used_mem ", used)
            print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | reserved_mem ", reserved)
            print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | available_mem ", total-reserved)
            sys.stdout.flush()
                                              
    if (cpu or ( chunk_size == 0 )):
        embs = [
            get_acts_and_grads(
                model=enc_learn.model,
                modules=module, 
                x=xb[0], 
                cpu=cpu
            )[0] 
            for xb in aux_dl
        ]
        if not cpu: embs=[emb.cpu() for emb in embs]
    else:
        embs = []
        total_chunks=max(1,round(len(X)/chunk_size))
        if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl len | " + str(len(X)) + " chunk size: " + str(chunk_size) + " => " + str(total_chunks) + " chunks")
        for i in range(0, total_chunks):
            if verbose > 0: 
                print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | Chunk [ " + str(i) + "/"+str(total_chunks)+"] => " + str(round(i*100/total_chunks)) + "%")
                sys.stdout.flush()
            chunk = [batch for (n, batch) in enumerate(aux_dl) if (chunk_size*i <= n  and chunk_size*(i+1) > n) ]
            chunk_embs = [
                get_acts_and_grads(
                    model=enc_learn.model,
                    modules=module,
                    x=xb[0], 
                    cpu=cpu
                )[0]
                for xb in chunk
            ]
            # Mueve los embeddings del bloque a la CPU
            chunk_embs = [emb.cpu() for emb in chunk_embs]
            embs.extend(chunk_embs)
            torch.cuda.empty_cache()
        if verbose > 0: 
            print("get_enc_embs_set_stride_set_batch_size | Get acts and grads | 100%")
            sys.stdout.flush()
    
    if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | concat embeddings")
    
    embs = to_concat(embs)
    
    if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | Reduce")
    
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    
    if verbose > 0: print("get_enc_embs_set_stride_set_batch_size | Convert to numpy")
    
    if to_numpy: 
        if cpu or chunk_size > 0:
            embs = embs.numpy() 
        else: 
            embs = embs.cpu().numpy()
            torch.cuda.empty_cache()
    if time_flag:
        t = time.time()-t_start
        if verbose > 0:
            print("get_enc_embs_set_stride_set_batch_size " + str(t) + " seconds -->")
        else:
            print("get_enc_embs_set_stride_set_batch_size " + str(t) + " seconds")
    if check_memory_usage: gpu_memory_status()
    if verbose > 0: 
        print("get_enc_embs_set_stride_set_batch_size -->")
    return embs

In [13]:
#| hide
import wandb
from dvats.utils import *
wandb_api = wandb.Api()
#enc_artifact = wandb_api.artifact('deepvats/mvp-SWV:latest')
#enc_learner = enc_artifact.to_obj()
#X = torch.rand(9, 1, 48)

In [ ]:
#| hide
#%time
#embs = get_enc_embs(X, enc_learner, cpu=True)
#test_eq(embs.shape[0], X.shape[0])
#embs.shape, embs.__class__

In [ ]:
#| hide
#%%time #TODO dont work with nb2py
#embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=False)
#test_eq(embs.shape[0], X.shape[0])
#embs.shape, embs.__class__, embs.device

In [ ]:
#| hide 
#%%time #TODO --> dont works with nb2py
#embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=True)
#test_eq(embs.shape[0], X.shape[0])
#embs.shape, embs.__class__